In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline
import matplotlib.pyplot as plt
import pylab

# This cell loads all previous model and create a stacker

In [3]:
# Function to load in all models.
def load_model(model_name, n_folds=10):
    ''' Input the model name to be loaded, and n_folds used.
    Returns the model that is aggregated from weights predicted from CV sets. '''
    train = []
    for i in range(n_folds):
        train.append(np.loadtxt('../model_4/{}_pred_fold_{}.txt'.format(model_name, i)))
    return train

if __name__ == '__main__':
    train = pd.read_csv('../train.csv')
    test = pd.read_csv('../test.csv')
    y_train = np.log1p(train.SalePrice)
    outliers_id = np.array([523, 1298])
    y_train = y_train.drop(outliers_id)

    ### Aggregate weights to be passed into layer 1 model
    # 1. This is Lasso predicted weights from Kfold training set
    train_lasso = load_model('ls', 10)
    train_lasso_folds = np.hstack((train_lasso[0], train_lasso[1], train_lasso[2], train_lasso[3], \
                                train_lasso[4], train_lasso[5], train_lasso[6], \
                                train_lasso[7], train_lasso[8], train_lasso[9]))
    print "\nChecking Lasso trainin set size: ", train_lasso_folds.shape

    rmse_check_1 = np.sqrt(mean_squared_error(np.log(train_lasso_folds), y_train))
    print "Lasso RMSE: ", rmse_check_1


    # 2. This is Ridge predicted weights from Kfold training set
    train_ridge = load_model('ridge', 10)
    train_ridge_folds = np.hstack((train_ridge[0], train_ridge[1], train_ridge[2], train_ridge[3], \
                                train_ridge[4], train_ridge[5], train_ridge[6], \
                                train_ridge[7], train_ridge[8], train_ridge[9]))
    print "\nChecking Ridge trainin set size: ", train_ridge_folds.shape

    rmse_check_2 = np.sqrt(mean_squared_error(np.log(train_ridge_folds), y_train))
    print "Ridge RMSE: ", rmse_check_2


    # 3. This is xgb predicted weights from Kfold training set
    train_xgb = load_model('xgb', 10)
    train_xgb_folds = np.hstack((train_xgb[0], train_xgb[1], train_xgb[2], train_xgb[3], \
                                train_xgb[4], train_xgb[5], train_xgb[6], \
                                train_xgb[7], train_xgb[8], train_xgb[9]))
    print "\nChecking XGB training set size: ", train_xgb_folds.shape

    rmse_check_3 = np.sqrt(mean_squared_error(np.log(train_xgb_folds), y_train))
    print "XGB RMSE: ", rmse_check_3


    # 4. This is knn predicted weights from Kfold training set
    train_knn = load_model('knn', 10)
    train_knn_folds = np.hstack((train_knn[0], train_knn[1], train_knn[2], train_knn[3], \
                                train_knn[4], train_knn[5], train_knn[6], \
                                train_knn[7], train_knn[8], train_knn[9]))
    print "\nChecking knn training set size: ", train_knn_folds.shape

    rmse_check_4 = np.sqrt(mean_squared_error(np.log(train_knn_folds), y_train))
    print "knn RMSE: ", rmse_check_4


    # 5. This is rf predicted weights from Kfold training set
    train_rf = load_model('rf', 10)
    train_rf_folds = np.hstack((train_rf[0], train_rf[1], train_rf[2], train_rf[3], \
                                train_rf[4], train_rf[5], train_rf[6], \
                                train_rf[7], train_rf[8], train_rf[9]))
    print "\nChecking rf training set size: ", train_rf_folds.shape

    rmse_check_5 = np.sqrt(mean_squared_error(np.log(train_rf_folds), y_train))
    print "rf RMSE: ", rmse_check_5

    ### Read in the prediction on test set. This will be test_x (test set features)
    xgb_pred = np.loadtxt('../model_4_test_set/xgb_test_pred.txt')
    lasso_pred = np.loadtxt('../model_4_test_set/lasso_test_pred.txt')
    ridge_pred = np.loadtxt('../model_4_test_set/ridge_test_pred.txt')
    rf_pred = np.loadtxt('../model_4_test_set/rf_test_pred.txt')
    knn_pred = np.loadtxt('../model_4_test_set/knn_test_pred.txt')


    # Resize the ridge and knn prediction so they can fit into the stacker.
    print '\n', type(xgb_pred), lasso_pred.shape, ridge_pred.shape, knn_pred.shape, rf_pred.shape
    # Stacking starts here.
    layer_1_train_x = np.vstack((train_xgb_folds, train_lasso_folds, train_ridge_folds, train_rf_folds, train_knn_folds)).T
    layer_1_test_x = np.vstack((xgb_pred, lasso_pred, ridge_pred, rf_pred, knn_pred)).T


    ### Use Random Forest to do the stacking
    rf_stack = RandomForestRegressor()
    rf_stack_rmse = np.sqrt(-cross_val_score(rf_stack, np.log(layer_1_train_x), y_train, cv=5, scoring='neg_mean_squared_error'))
    print "\nRF Stacker CV RMSE: ", (rf_stack_rmse.mean())

    rf_stack.fit(np.log(layer_1_train_x), y_train)
    rf_stack_final_pred = rf_stack.predict(layer_1_test_x)
    df_rf_stack_final_pred = pd.DataFrame(np.exp(rf_stack_final_pred), index=test["Id"], columns=["SalePrice"])
    df_rf_stack_final_pred.to_csv('submission13_rf_stack.csv', header=True, index_label='Id')
    print "\n", df_rf_stack_final_pred.head()


    ### User Linear Regression to do the stacking
    lr = LinearRegression()
    lr_rmse = np.sqrt(-cross_val_score(lr, np.log(layer_1_train_x), y_train, cv=5, scoring='neg_mean_squared_error'))
    print "\nLinear Stacker CV RMSE: ", (lr_rmse.mean())

    lr.fit(np.log(layer_1_train_x), y_train)
    final_pred = lr.predict(layer_1_test_x)
    df_final_pred = pd.DataFrame(np.exp(final_pred), index=test["Id"], columns=["SalePrice"])
    df_final_pred.to_csv('submission12_linear_stack.csv', header=True, index_label='Id')
    print "\n", df_final_pred.head()


Checking Lasso trainin set size:  (1458,)
Lasso RMSE:  0.112229412799

Checking Ridge trainin set size:  (1458,)
Ridge RMSE:  0.117605942387

Checking XGB training set size:  (1458,)
XGB RMSE:  0.117635216797

Checking knn training set size:  (1458,)
knn RMSE:  0.167952442091

Checking rf training set size:  (1458,)
rf RMSE:  0.132003171424

<type 'numpy.ndarray'> (1459,) (1459,) (1459,) (1459,)

RF Stacker CV RMSE:  0.12311867718

          SalePrice
Id                 
1461  129207.666221
1462  171987.873184
1463  172548.736950
1464  204578.648897
1465  199166.941510

Linear Stacker CV RMSE:  0.110599566173

          SalePrice
Id                 
1461  120558.833097
1462  162124.185112
1463  178508.650558
1464  198781.608529
1465  199234.023798


/Library/Python/2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [10]:
import xgboost as xgb
from xgboost import XGBRegressor

# Grid Search with XGBoost. Tuning max_depth & min_child_weight
xgb_model = XGBRegressor()
xgb_params = {'max_depth': [1], \
              'subsample': [0.7], \
              'n_estimators': [5000], \
              'learning_rate': [0.01], \
              'colsample_bytree': [0.6], \
              'min_child_weight': [6], \
              'gamma': [0], \
              'scale_pos_weight': [1], \
              'reg_lambda':[0.9], \
              'seed': [42]
             }
### Tried hyperparameters#########
# max_depth = [2, 3, 6, 9]
# n_estimators = [500]
# min_child_weight = [1.25, 3, 5, 7, 9]
##################################

xgb_grid = GridSearchCV(xgb_model, xgb_params, cv=10, verbose=1, scoring='neg_mean_squared_error')
xgb_grid.fit(np.log(layer_1_train_x), y_train)
print("Score: ", xgb_grid.cv_results_)
print("mean_test_score: ", xgb_grid.cv_results_['mean_test_score'])
mean_test_score = xgb_grid.cv_results_['mean_test_score'].mean()
print("Average mean_test_score: ", mean_test_score)


### Prediction from stacker is here.
xgb_stack_final_pred = xgb_grid.predict(layer_1_test_x)
df_xgb_stack_final_pred = pd.DataFrame(np.exp(xgb_stack_final_pred), index=test["Id"], columns=["SalePrice"])
df_xgb_stack_final_pred.to_csv('submission14_xgb_stack.csv', header=True, index_label='Id')

### Print the best param and CV score here.
print(xgb_grid.best_params_)
print '\n', (xgb_grid.best_score_, np.sqrt(-xgb_grid.best_score_))

Fitting 10 folds for each of 6 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:  1.9min finished


('Score: ', {'param_scale_pos_weight': masked_array(data = [1 1 1 1 1 1],
             mask = [False False False False False False],
       fill_value = ?)
, 'rank_test_score': array([1, 2, 3, 5, 4, 6], dtype=int32), 'param_gamma': masked_array(data = [0 0 0 0 0 0],
             mask = [False False False False False False],
       fill_value = ?)
, 'split6_test_score': array([-0.01421315, -0.01420057, -0.01410908, -0.0147819 , -0.0147802 ,
       -0.01464187]), 'split7_train_score': array([-0.00992227, -0.01000877, -0.01011812, -0.007294  , -0.00757964,
       -0.0078914 ]), 'split0_train_score': array([-0.00923204, -0.00927174, -0.00936809, -0.00685629, -0.00704583,
       -0.00733514]), 'split2_test_score': array([-0.01028112, -0.01025563, -0.01016041, -0.01068304, -0.01059665,
       -0.0104838 ]), 'mean_fit_time': array([ 1.42232699,  1.48049843,  1.53980062,  2.33758044,  2.1454263 ,
        2.14090152]), 'split3_train_score': array([-0.00938455, -0.00943336, -0.00951781, -0.00677